<a href="https://colab.research.google.com/github/TheDeepHub/ParkingDetector/blob/main/Train_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
import os

# Specify the path to your zip file
zip_path = '/content/clf-data.zip'  # Change this to the path of your zip file

# Specify the directory to extract the contents into
extract_to = '/content/parking_dataset'  # Change this to your desired output directory

# Create the directory if it doesn't exist
if not os.path.exists(extract_to):
    os.makedirs(extract_to)

# Unzipping the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Contents extracted to {extract_to}")


Contents extracted to /content/parking_dataset


Before defining how we will be importing the images into TensorFlow, we´ll make a quick analysis of the images dimensions. They don´t have a standard size.

In [2]:
import os
import tensorflow as tf

def analyze_image_sizes(directory_path):
    image_sizes = {}
    for class_name in os.listdir(directory_path):
        class_path = os.path.join(directory_path, class_name)
        if not os.path.isdir(class_path):
            continue
        sizes = []
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            image = tf.io.read_file(image_path)
            image = tf.image.decode_image(image, channels=3, expand_animations=False)
            sizes.append(image.shape[:2])  # (height, width)
        image_sizes[class_name] = sizes

    return image_sizes

def summarize_sizes(image_sizes):
    all_sizes = []
    for class_name, sizes in image_sizes.items():
        sizes.sort(key=lambda x: (x[0] * x[1]), reverse=True)  # Sort by area
        print(f"Class '{class_name}':")
        print(f"  5 Largest: {sizes[:5]}")
        print(f"  5 Smallest: {sizes[-5:]}")
        avg_height = sum(size[0] for size in sizes) / len(sizes)
        avg_width = sum(size[1] for size in sizes) / len(sizes)
        print(f"  Average Size: {(avg_height, avg_width)}")
        all_sizes.extend(sizes)

    # Global stats
    all_sizes.sort(key=lambda x: (x[0] * x[1]), reverse=True)  # Sort by area
    print("Overall:")
    print(f"  5 Largest: {all_sizes[:5]}")
    print(f"  5 Smallest: {all_sizes[-5:]}")
    avg_height = sum(size[0] for size in all_sizes) / len(all_sizes)
    avg_width = sum(size[1] for size in all_sizes) / len(all_sizes)
    print(f"  Average Size: {(avg_height, avg_width)}")

directory_path = '/content/parking_dataset/clf-data'  # Path to your dataset directory
image_sizes = analyze_image_sizes(directory_path)
summarize_sizes(image_sizes)

Class 'empty':
  5 Largest: [TensorShape([36, 65]), TensorShape([36, 65]), TensorShape([36, 65]), TensorShape([36, 65]), TensorShape([36, 65])]
  5 Smallest: [TensorShape([26, 62]), TensorShape([26, 62]), TensorShape([26, 62]), TensorShape([26, 62]), TensorShape([26, 62])]
  Average Size: (28.945484400656813, 67.66863711001642)
Class 'not_empty':
  5 Largest: [TensorShape([33, 69]), TensorShape([33, 69]), TensorShape([33, 69]), TensorShape([33, 69]), TensorShape([33, 69])]
  5 Smallest: [TensorShape([26, 61]), TensorShape([26, 61]), TensorShape([26, 61]), TensorShape([26, 61]), TensorShape([26, 61])]
  Average Size: (28.770443349753695, 68.21182266009852)
Overall:
  5 Largest: [TensorShape([36, 65]), TensorShape([36, 65]), TensorShape([36, 65]), TensorShape([36, 65]), TensorShape([36, 65])]
  5 Smallest: [TensorShape([26, 61]), TensorShape([26, 61]), TensorShape([26, 61]), TensorShape([26, 61]), TensorShape([26, 61])]
  Average Size: (28.857963875205254, 67.94022988505748)


In [3]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2)

In [4]:
dataset_path = "/content/parking_dataset/clf-data"
train_ds = data_gen.flow_from_directory(
    directory=dataset_path,
    subset="training",
    seed=123,
    target_size=(29,68),
    batch_size=32,
    class_mode='sparse',
    shuffle=True
)

val_ds = data_gen.flow_from_directory(
    directory=dataset_path,
    subset="validation",
    seed=123,
    target_size=(29, 68),
    batch_size=32,
    class_mode='sparse',
    shuffle=True
)

Found 4872 images belonging to 2 classes.
Found 1218 images belonging to 2 classes.


In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define your model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(29, 68, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # Change the final layer to match the number of classes in your dataset
])


In [20]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

Epoch 1/10
153/153 [==============================] - 15s 89ms/step - loss: 0.1242 - accuracy: 0.9583 - val_loss: 0.1087 - val_accuracy: 0.9688
Epoch 2/10
153/153 [==============================] - 13s 88ms/step - loss: 0.0068 - accuracy: 0.9982 - val_loss: 0.0708 - val_accuracy: 0.9828
Epoch 3/10
153/153 [==============================] - 13s 87ms/step - loss: 0.0112 - accuracy: 0.9961 - val_loss: 0.0458 - val_accuracy: 0.9836
Epoch 4/10
153/153 [==============================] - 14s 89ms/step - loss: 0.0044 - accuracy: 0.9986 - val_loss: 0.0237 - val_accuracy: 0.9901
Epoch 5/10
153/153 [==============================] - 13s 87ms/step - loss: 4.7421e-04 - accuracy: 1.0000 - val_loss: 0.0324 - val_accuracy: 0.9885
Epoch 6/10
153/153 [==============================] - 13s 87ms/step - loss: 1.2796e-04 - accuracy: 1.0000 - val_loss: 0.0257 - val_accuracy: 0.9885
Epoch 7/10
153/153 [==============================] - 13s 87ms/step - loss: 5.9993e-05 - accuracy: 1.0000 - val_loss: 0.0270 - v

In [22]:
model.save('/content/model/path_to_my_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.7 MB/s eta 0:00:00


In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from kerastuner import HyperModel

class MyHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = Sequential()
        model.add(Conv2D(filters=hp.Int('conv_1_filters', min_value=32, max_value=128, step=32),
                         kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
                         activation='relu', input_shape=self.input_shape))
        model.add(MaxPooling2D(2, 2))
        model.add(Conv2D(filters=hp.Int('conv_2_filters', min_value=64, max_value=256, step=64),
                         kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
                         activation='relu'))
        model.add(MaxPooling2D(2, 2))
        model.add(Flatten())
        model.add(Dense(units=hp.Int('dense_units', min_value=64, max_value=512, step=64),
                        activation='relu'))
        model.add(Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
        model.add(Dense(self.num_classes, activation='softmax'))

        model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        return model

In [ ]:
from kerastuner.tuners import RandomSearch

input_shape = (29, 68, 3)  # Adjust based on your dataset
num_classes = 2  # Adjust based on your dataset

hypermodel = MyHyperModel(input_shape=input_shape, num_classes=num_classes)

tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='/content/my_dir/hparam_tuning',
    project_name='hparam_tuning'
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

stop_early = EarlyStopping(monitor='val_loss', patience=5)

tuner.search(train_ds,
             epochs=10,
             validation_data=val_ds,
             callbacks=[stop_early])

Trial 7 Complete [00h 11m 27s]
val_accuracy: 0.9778324961662292

Best val_accuracy So Far: 0.9819375872612
Total elapsed time: 00h 52m 49s

Search: Running Trial #8

Value             |Best Value So Far |Hyperparameter
64                |64                |conv_1_filters
5                 |5                 |conv_1_kernel
256               |192               |conv_2_filters
3                 |3                 |conv_2_kernel
192               |192               |dense_units
0.4               |0.3               |dropout
0.001             |0.001             |learning_rate

Epoch 1/10
153/153 [==============================] - 54s 344ms/step - loss: 1.6971 - accuracy: 0.9540 - val_loss: 0.1104 - val_accuracy: 0.9770
Epoch 2/10
153/153 [==============================] - 48s 313ms/step - loss: 0.0032 - accuracy: 0.9994 - val_loss: 0.1665 - val_accuracy: 0.9770
Epoch 3/10
153/153 [==============================] - 50s 329ms/step - loss: 4.1169e-04 - accuracy: 0.9998 - val_loss: 0.2190 - val_